## 00- ETL : Get NDX, SP500, Gold, BTCUSD and ETHUSD candlesticks 

Note: BTCUSD and ETHUSD candlesticks are obtained from cryptocompare.com

This section focuses on retrieving Bitcoin and Ether's OHLCV time series from cryptocompare.com using the ETL process. 

The main function responsible for this task is 'get_crypto_OHLCV_cc()'. 

**get_crypto_OHLCV_cc()**
> - The function checks for an existing .csv file in the 'Resources' folder and then compares the latest information extracted to complete the .csv with the last available information. 
> - The resulting merged dataset is then saved in the 'Resources' folder. 
> - It can be used by any crypto pair available on cryptocompare.com

*To-do:* 
> - IMPROVEMENT: instead of using a .csv file as a repository, it would be better to use a sqlite or mongodb database.

*Sources:* 
> - https://towardsdatascience.com/obtaining-historical-and-real-time-crypto-data-with-very-simple-web-programming-7b481f153630

In [3]:
# Import Dependencies

# Libraries for data handling and time manipulation
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import pytz

# Libraries for handling system environment files and variables
import os
from pathlib import Path
from dotenv import load_dotenv

# Libraries for API calls
import requests  # HTTP library
import json      # JSON handling library

# Custom function from the functions_libraryfolder
from functions_library.functions import DFinfo


In [4]:

load_dotenv()
ccompare_api_key = os.getenv('CCOMPARE_API_KEY')


In [5]:
# Function to get crypto OHLCV data from CryptoCompare

def get_crypto_OHLCV_cc(crypto='ETH', fiat='USD', limit =2000, toTS=-1,api_key = ccompare_api_key):
    url = f"https://min-api.cryptocompare.com/data/v2/histohour?fsym={crypto}&tsym={fiat}&limit={limit}&toTs={toTS}&api_key={api_key}"

    r = requests.get(url, headers={"User-Agent": "XY"})

    if r.status_code == 200:
        with open(f"resources/{crypto}{fiat}_ccompare.json", "wb") as file:
            file.write(r.content)
        print(f"{crypto}-{fiat} JSON data downloaded successfully.")
    else:
        print(f"Failed to download {crypto}-{fiat} data.")
        print(r)

    # # Load the JSON data from the file
    pair_json = Path(f"resources/{crypto}{fiat}_ccompare.json")
    list_pair_df = pd.read_json(pair_json)
    ohlcv_list = list_pair_df['Data']['Data']

    # Create a DataFrame
    pair_1H_df = pd.DataFrame(ohlcv_list)

    # Convert 'time' column to datetime format
    pair_1H_df['timestamp'] = pd.to_datetime(pair_1H_df['time'], unit='s')
    pair_1H_df.drop(columns=['time'], inplace=True)

    # Set 'time' as the index
    pair_1H_df.set_index('timestamp', inplace=True)
    pair_1H_df.index = pair_1H_df.index.tz_localize('UTC')

    # Rename columns to match OHLCV format
    pair_1H_df.rename(columns={'volumefrom': 'volume'}, inplace=True)

    # Reorder columns
    pair_1H_df = pair_1H_df[['open', 'high', 'low', 'close', 'volume']]

    return pair_1H_df


In [6]:
test = get_crypto_OHLCV_cc(crypto='ETH', fiat='USD', limit =2000, toTS=-1,api_key = ccompare_api_key)
DFinfo(test)


ETH-USD JSON data downloaded successfully.


,open,high,low,close,volume
timestamp,,,,,
2024-11-26 15:00:00+00:00,3345.72,3354.47,3296.83,3297.27,90949.71
2024-11-26 16:00:00+00:00,3297.27,3339.26,3292.16,3336.20,64494.00
2024-11-26 17:00:00+00:00,3336.20,3337.54,3309.18,3326.93,46906.44


,open,high,low,close,volume
timestamp,,,,,
2025-02-17 21:00:00+00:00,2733.06,2778.12,2732.21,2776.63,19787.13
2025-02-17 22:00:00+00:00,2776.63,2777.54,2733.27,2737.98,9499.29
2025-02-17 23:00:00+00:00,2737.98,2751.13,2734.71,2743.99,13211.21


In [7]:
# Function to download and merge crypto OHLCV data from CryptoCompare

def download_and_merge_crypto_OHLCV_cc(crypto='ETH', fiat='USD'):
    # initialize variables and list to store dataframes
    limit=2000

    # Convert the current time to UTC
    now_utc = datetime.now(pytz.timezone('UTC'))

    # Check if the CSV file exists
    csv_path = f'Resources/{crypto}{fiat}_1H_ccompare.csv'

    if Path(csv_path).is_file():

        # Load the CSV file into a DataFrame
        result_df = pd.read_csv(csv_path, parse_dates=['timestamp'], index_col='timestamp')

        # Convert the latest timestamp to UTC
        latest_timestamp = result_df.index.max().tz_convert('UTC')
        first_timestamp = result_df.index.min().tz_convert('UTC')

        # Calculate the number of hours between now (in UTC) and the latest entry in the CSV file (in UTC)
        hours_since_latest = (now_utc - latest_timestamp).total_seconds() / 3600
        total_batches = int(hours_since_latest / limit) + 1
        hours_to_download = int(hours_since_latest)

        # Print information about the CSV file
        print(f"Detecting OHLCV data that was previously downloaded:")
        print(f" > latest timestamp available (UTC): {latest_timestamp}")
        print(f" > first timestamp available (UTC): {first_timestamp}")
        print("")

    else:
        # Initialize an empty DataFrame if the CSV file does not exist
        result_df = pd.DataFrame()

        # Calculate the number of batches needed from January 2017 until now
        start_date = datetime(2017, 1, 1, 0, 0, 0, tzinfo=pytz.UTC)
        hours_since_start = (now_utc - start_date).total_seconds() / 3600
        hours_since_start = (now_utc - start_date).total_seconds() / 3600
        total_batches = int(hours_since_start / limit) + 1
        hours_to_download = int(hours_since_start)

    if hours_to_download == 0:
        print(f"Already up to date! No new {crypto}-{fiat} data to download.")

    else:
        print(f"Downloading {total_batches} batches of {limit} hours each from CryptoCompare.")
        print(f"for completing the {hours_to_download} missing hours of {crypto}-{fiat}")

        # We will start downloading from the latest timestamp available in the API (-1)
        batch_timestamp = -1

        for batch in range(total_batches):
            # Download data for the current batch
            df_batch = get_crypto_OHLCV_cc(crypto=crypto, fiat=fiat, limit=limit, toTS=batch_timestamp, api_key=ccompare_api_key)

            # Append the new batch to the DataFrame
            result_df = pd.concat([result_df, df_batch])

            # Check if the batch was downloaded successfully
            if not df_batch.empty:
                # Update the timestamp for the next batch
                earliest_timestamp = df_batch.index.min()
                latest_timestamp = df_batch.index.max()

                # Convert the datetime object to Unix time
                earliest_timestamp_unix = int(time.mktime(earliest_timestamp.timetuple()))
                batch_timestamp = earliest_timestamp_unix
                print(f"Batch {batch + 1}/{total_batches} downloaded successfully.")

            else:
                print(f"Failed to download batch {batch + 1}.")

    print(f"Latest {crypto}-{fiat} OHLCV dataset updated.")

    # Remove rows with missing values
    result_df.dropna(inplace=True)

    # Sort by timestamp and remove duplicates
    result_df = result_df.sort_index().drop_duplicates()

    # Checking if there is any misisng time steps
    ### Create a complete datetime index with 1-hour frequency
    complete_index = pd.date_range(start=result_df.index.min(), end=result_df.index.max(), freq='H')

    ### Find the difference between the complete index and the existing index
    missing_timestamps = complete_index.difference(result_df.index)

    if missing_timestamps.empty:
        print("No missing timestamps found.")

        # Save the resulting dataframe as both .csv and .json
        # When saving to CSV, reset the index
        result_df.reset_index().to_csv(csv_path, header=True, index=False)
        result_df.to_json(f'Resources/{crypto}{fiat}_1H_ccompare.json', orient='records', date_format='iso')

        print(f"All the following {crypto}-{fiat} OHLCV info has been saved to disk and is available now =)")
        DFinfo(result_df)

    else:
        print(f"{len(missing_timestamps)} missing timestamps found. Aborting - Please try again.")

    return result_df


In [11]:
# Download and merge ETH-USD data from CryptoCompare
ETHUSD_dataset = download_and_merge_crypto_OHLCV_cc(crypto='ETH', fiat='USD')

Detecting OHLCV data that was previously downloaded:
 > latest timestamp available (UTC): 2025-02-17 23:00:00+00:00
 > first timestamp available (UTC): 2016-11-01 05:00:00+00:00

Already up to date! No new ETH-USD data to download.
Latest ETH-USD OHLCV dataset updated.
No missing timestamps found.


/var/folders/nb/f4s8fppj0j3gshk7xfn2cwf00000gn/T/ipykernel_54275/3447939821.py:85: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  complete_index = pd.date_range(start=result_df.index.min(), end=result_df.index.max(), freq='H')


All the following ETH-USD OHLCV info has been saved to disk and is available now =)


,open,high,low,close,volume
timestamp,,,,,
2016-11-01 05:00:00+00:00,10.65,10.75,10.61,10.74,12279.18
2016-11-01 06:00:00+00:00,10.74,10.79,10.65,10.67,19010.58
2016-11-01 07:00:00+00:00,10.67,10.82,10.61,10.62,49508.65


,open,high,low,close,volume
timestamp,,,,,
2025-02-17 21:00:00+00:00,2733.06,2778.12,2732.21,2776.63,19787.13
2025-02-17 22:00:00+00:00,2776.63,2777.54,2733.27,2737.98,9499.29
2025-02-17 23:00:00+00:00,2737.98,2751.13,2734.71,2744.14,13218.26


In [12]:
ETHUSD_dataset

,open,high,low,close,volume
timestamp,,,,,
2016-11-01 05:00:00+00:00,10.65,10.75,10.61,10.74,12279.18
2016-11-01 06:00:00+00:00,10.74,10.79,10.65,10.67,19010.58
2016-11-01 07:00:00+00:00,10.67,10.82,10.61,10.62,49508.65
2016-11-01 08:00:00+00:00,10.62,10.66,10.47,10.50,101202.05
2016-11-01 09:00:00+00:00,10.50,10.75,10.45,10.51,76259.74
...,...,...,...,...,...
2025-02-17 19:00:00+00:00,2700.92,2722.32,2687.29,2718.33,34783.40
2025-02-17 20:00:00+00:00,2718.33,2746.23,2716.75,2733.06,20680.18
2025-02-17 21:00:00+00:00,2733.06,2778.12,2732.21,2776.63,19787.13


In [13]:
# Download and merge ETH-USD data from CryptoCompare
BTCUSD_dataset = download_and_merge_crypto_OHLCV_cc(crypto='BTC', fiat='USD')


Detecting OHLCV data that was previously downloaded:
 > latest timestamp available (UTC): 2024-12-03 15:00:00+00:00
 > first timestamp available (UTC): 2016-10-24 07:00:00+00:00

for completing the 1832 missing hours of BTC-USD
BTC-USD JSON data downloaded successfully.
Batch 1/1 downloaded successfully.
Latest BTC-USD OHLCV dataset updated.
No missing timestamps found.


/var/folders/nb/f4s8fppj0j3gshk7xfn2cwf00000gn/T/ipykernel_54275/3447939821.py:85: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  complete_index = pd.date_range(start=result_df.index.min(), end=result_df.index.max(), freq='H')


All the following BTC-USD OHLCV info has been saved to disk and is available now =)


,open,high,low,close,volume
timestamp,,,,,
2016-10-24 07:00:00+00:00,651.41,651.91,647.62,648.31,892.71
2016-10-24 08:00:00+00:00,648.31,651.51,648.01,650.41,1730.28
2016-10-24 09:00:00+00:00,650.41,649.60,646.81,647.14,1803.12


,open,high,low,close,volume
timestamp,,,,,
2025-02-17 21:00:00+00:00,95991.55,96433.50,95939.87,96420.75,746.31
2025-02-17 22:00:00+00:00,96420.75,96431.40,95870.56,95900.64,330.13
2025-02-17 23:00:00+00:00,95900.64,96023.85,95719.22,95787.10,603.35


In [14]:
# display the resulting dataframe
BTCUSD_dataset

,open,high,low,close,volume
timestamp,,,,,
2016-10-24 07:00:00+00:00,651.41,651.91,647.62,648.31,892.71
2016-10-24 08:00:00+00:00,648.31,651.51,648.01,650.41,1730.28
2016-10-24 09:00:00+00:00,650.41,649.60,646.81,647.14,1803.12
2016-10-24 10:00:00+00:00,647.14,649.30,646.13,648.09,1557.74
2016-10-24 11:00:00+00:00,648.09,652.81,648.74,651.84,1320.71
...,...,...,...,...,...
2025-02-17 19:00:00+00:00,95345.00,95607.30,95226.61,95547.29,856.13
2025-02-17 20:00:00+00:00,95547.29,96283.82,95512.31,95991.55,863.67
2025-02-17 21:00:00+00:00,95991.55,96433.50,95939.87,96420.75,746.31


In [43]:
# Import Dependencies
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import pytz
import os
from pathlib import Path
import yfinance as yf

def get_market_OHLCV_yf(symbol='^GSPC', start='2016-10-24'):
    """
    Get daily market data from Yahoo Finance
    symbol: '^GSPC' for S&P 500 or '^NDX' for NASDAQ 100
    start: start date in 'YYYY-MM-DD' format
    """
    try:
        # Download daily data
        ticker = yf.Ticker(symbol)
        df = ticker.history(start=start, interval='1d')
        
        if df.empty:
            print(f"No data retrieved for {symbol}")
            return None
            
        # Keep only OHLCV columns
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
        
        # Rename columns to match our format
        df.columns = df.columns.str.lower()
        
        # Make sure index is UTC
        if df.index.tz is None:
            df.index = df.index.tz_localize('UTC')
        elif df.index.tz != pytz.UTC:
            df.index = df.index.tz_convert('UTC')
            
        print(f"{symbol} data downloaded successfully.")
        return df
        
    except Exception as e:
        print(f"Error downloading {symbol} data: {str(e)}")
        return None


import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import pytz
import os
from pathlib import Path
import yfinance as yf

def get_market_OHLCV_yf(symbol='^GSPC', start='2016-10-24'):
    """
    Get daily market data from Yahoo Finance
    symbol: '^GSPC' for S&P 500 or '^NDX' for NASDAQ 100
    start: start date in 'YYYY-MM-DD' format
    """
    try:
        # Download daily data
        ticker = yf.Ticker(symbol)
        df = ticker.history(start=start, interval='1d')
        
        if df.empty:
            print(f"No data retrieved for {symbol}")
            return None
            
        # Keep only OHLCV columns
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
        
        # Rename columns to match our format
        df.columns = df.columns.str.lower()
        
        # Make sure index is UTC
        if df.index.tz is None:
            df.index = df.index.tz_localize('UTC')
        elif df.index.tz != pytz.UTC:
            df.index = df.index.tz_convert('UTC')
            
        print(f"{symbol} data downloaded successfully.")
        return df
        
    except Exception as e:
        print(f"Error downloading {symbol} data: {str(e)}")
        return None

def download_and_merge_market_OHLCV_yf(symbol='^GSPC', start='2016-10-24'):
    """
    Download and merge daily market OHLCV data from Yahoo Finance
    Handles existing CSV files and updates with new data
    """
    # Convert symbol to a filename-friendly format
    filename = symbol.replace('^', '')
    csv_path = f'Resources/{filename}_1D_yfinance.csv'
    
    if Path(csv_path).is_file():
        # Load existing data
        result_df = pd.read_csv(csv_path, parse_dates=['Date'], index_col='Date')  # Changed from 'timestamp' to 'Date'
        if result_df.index.tz is None:
            result_df.index = result_df.index.tz_localize('UTC')
        
        latest_timestamp = result_df.index.max()
        first_timestamp = result_df.index.min()
        
        print(f"Detecting OHLCV data that was previously downloaded:")
        print(f" > latest timestamp available (UTC): {latest_timestamp}")
        print(f" > first timestamp available (UTC): {first_timestamp}")
        print("")
        
        # Get new data from the last available date
        new_data = get_market_OHLCV_yf(symbol=symbol, start=latest_timestamp.strftime('%Y-%m-%d'))
        
        if new_data is not None:
            # Combine existing and new data
            result_df = pd.concat([result_df, new_data])
            # Remove duplicates and sort
            result_df = result_df[~result_df.index.duplicated(keep='last')].sort_index()
    else:
        # Get all available data if no CSV exists
        result_df = get_market_OHLCV_yf(symbol=symbol, start=start)
        
        if result_df is None:
            return None
    
    # Remove any NaN values
    result_df.dropna(inplace=True)
    
    # Create a complete business day index
    full_index = pd.date_range(start=result_df.index.min(), 
                             end=result_df.index.max(), 
                             freq='B',  # Business days
                             tz='UTC')
    
    # Find missing days
    missing_days = full_index.difference(result_df.index)
    
    if len(missing_days) > 0:
        print(f"\nNote: Found {len(missing_days)} missing business days.")
        print("These might be holidays or other non-trading days.")
        print("First few missing days:", missing_days[:5].strftime('%Y-%m-%d').tolist())
    else:
        print("\nNo missing business days found.")
    
    # Save to CSV
    result_df.reset_index().to_csv(csv_path, index=False)  # 'Date' column will be preserved
    print(f"\nOHLCV data saved to {csv_path}")
    
    # Save to JSON
    json_path = f'Resources/{filename}_1D_yfinance.json'
    result_df.reset_index().to_json(json_path, orient='records', date_format='iso')
    print(f"OHLCV data saved to {json_path}")
    
    print(f"\nAll {symbol} OHLCV info has been saved to disk and is available now =)")
    print("\nFirst few rows:")
    print(result_df.head())
    print("\nLast few rows:")
    print(result_df.tail())
    print("\nDataset Info:")
    print(result_df.info())
    
    return result_df
# Example usage:
if __name__ == "__main__":
    # Download S&P 500 data
    sp500_data = download_and_merge_market_OHLCV_yf(symbol='^GSPC', start='2016-10-24')
    
    # Download NASDAQ 100 data
    nasdaq_data = download_and_merge_market_OHLCV_yf(symbol='^NDX', start='2016-10-24')

# Example usage:
if __name__ == "__main__":
    # Download S&P 500 data
    sp500_data = download_and_merge_market_OHLCV_yf(symbol='^GSPC', start='2016-10-24')
    
    # Download NASDAQ 100 data
    nasdaq_data = download_and_merge_market_OHLCV_yf(symbol='^NDX', start='2016-10-24')

Detecting OHLCV data that was previously downloaded:
 > latest timestamp available (UTC): 2025-02-14 05:00:00+00:00
 > first timestamp available (UTC): 2016-10-24 04:00:00+00:00

^GSPC data downloaded successfully.

Note: Found 835 missing business days.
These might be holidays or other non-trading days.
First few missing days: ['2016-11-07', '2016-11-08', '2016-11-09', '2016-11-10', '2016-11-11']

OHLCV data saved to Resources/GSPC_1D_yfinance.csv
OHLCV data saved to Resources/GSPC_1D_yfinance.json

All ^GSPC OHLCV info has been saved to disk and is available now =)

First few rows:
                                  open         high          low        close  \
Date                                                                            
2016-10-24 04:00:00+00:00  2148.500000  2154.790039  2146.909912  2151.330078   
2016-10-25 04:00:00+00:00  2149.719971  2151.439941  2141.929932  2143.159912   
2016-10-26 04:00:00+00:00  2136.969971  2145.729980  2131.590088  2139.429932   
2016

In [44]:
sp500_data =download_and_merge_market_OHLCV_yf(symbol='^GSPC', start='2016-10-24')

Detecting OHLCV data that was previously downloaded:
 > latest timestamp available (UTC): 2025-02-14 05:00:00+00:00
 > first timestamp available (UTC): 2016-10-24 04:00:00+00:00

^GSPC data downloaded successfully.

Note: Found 835 missing business days.
These might be holidays or other non-trading days.
First few missing days: ['2016-11-07', '2016-11-08', '2016-11-09', '2016-11-10', '2016-11-11']

OHLCV data saved to Resources/GSPC_1D_yfinance.csv
OHLCV data saved to Resources/GSPC_1D_yfinance.json

All ^GSPC OHLCV info has been saved to disk and is available now =)

First few rows:
                                  open         high          low        close  \
Date                                                                            
2016-10-24 04:00:00+00:00  2148.500000  2154.790039  2146.909912  2151.330078   
2016-10-25 04:00:00+00:00  2149.719971  2151.439941  2141.929932  2143.159912   
2016-10-26 04:00:00+00:00  2136.969971  2145.729980  2131.590088  2139.429932   
2016

In [45]:
sp500_data

,open,high,low,close,volume
Date,,,,,
2016-10-24 04:00:00+00:00,2148.500000,2154.790039,2146.909912,2151.330078,3359950000
2016-10-25 04:00:00+00:00,2149.719971,2151.439941,2141.929932,2143.159912,3756200000
2016-10-26 04:00:00+00:00,2136.969971,2145.729980,2131.590088,2139.429932,3778120000
2016-10-27 04:00:00+00:00,2144.060059,2147.129883,2132.520020,2133.040039,4209400000
2016-10-28 04:00:00+00:00,2132.229980,2140.719971,2119.360107,2126.409912,4028270000
...,...,...,...,...,...
2025-02-10 05:00:00+00:00,6046.399902,6073.379883,6044.839844,6066.439941,4458760000
2025-02-11 05:00:00+00:00,6049.319824,6076.279785,6042.339844,6068.500000,4324880000
2025-02-12 05:00:00+00:00,6025.080078,6063.000000,6003.000000,6051.970215,4627960000


In [46]:
ndx100=download_and_merge_market_OHLCV_yf(symbol='^NDX', start='2016-10-24')

Detecting OHLCV data that was previously downloaded:
 > latest timestamp available (UTC): 2025-02-14 05:00:00+00:00
 > first timestamp available (UTC): 2016-10-24 04:00:00+00:00

^NDX data downloaded successfully.

Note: Found 835 missing business days.
These might be holidays or other non-trading days.
First few missing days: ['2016-11-07', '2016-11-08', '2016-11-09', '2016-11-10', '2016-11-11']

OHLCV data saved to Resources/NDX_1D_yfinance.csv
OHLCV data saved to Resources/NDX_1D_yfinance.json

All ^NDX OHLCV info has been saved to disk and is available now =)

First few rows:
                                  open         high          low        close  \
Date                                                                            
2016-10-24 04:00:00+00:00  4884.160156  4911.589844  4880.770020  4909.970215   
2016-10-25 04:00:00+00:00  4908.180176  4911.759766  4885.899902  4891.450195   
2016-10-26 04:00:00+00:00  4863.520020  4886.319824  4846.439941  4860.589844   
2016-10-

In [34]:
ndx100

,open,high,low,close,volume
Date,,,,,
2016-10-24 04:00:00+00:00,4884.160156,4911.589844,4880.770020,4909.970215,1533370000
2016-10-25 04:00:00+00:00,4908.180176,4911.759766,4885.899902,4891.450195,1580830000
2016-10-26 04:00:00+00:00,4863.520020,4886.319824,4846.439941,4860.589844,1732300000
2016-10-27 04:00:00+00:00,4883.069824,4885.830078,4831.689941,4836.450195,1926820000
2016-10-28 04:00:00+00:00,4820.500000,4848.549805,4794.069824,4805.589844,1911810000
...,...,...,...,...,...
2025-02-10 05:00:00+00:00,21670.060547,21793.130859,21645.169922,21756.730469,9535440000
2025-02-11 05:00:00+00:00,21629.109375,21776.250000,21625.509766,21693.519531,9269380000
2025-02-12 05:00:00+00:00,21475.410156,21745.660156,21454.189453,21719.259766,7946550000


In [47]:
def download_and_merge_gold_OHLCV_yf(symbol='GC=F', start='2016-10-24'):
    """
    Download and merge daily gold OHLCV data from Yahoo Finance
    Handles existing CSV files and updates with new data
    """
    try:
        # Convert symbol to a filename-friendly format
        filename = 'GOLD'  # Using GOLD instead of GC=F for filename
        csv_path = f'Resources/{filename}_1D_yfinance.csv'
        
        if Path(csv_path).is_file():
            # Load existing data with proper date parsing
            result_df = pd.read_csv(csv_path)
            result_df['Date'] = pd.to_datetime(result_df['Date'])
            result_df.set_index('Date', inplace=True)
            
            # Handle timezone
            if result_df.index.tz is None:
                result_df.index = result_df.index.tz_localize('UTC')
            
            latest_timestamp = result_df.index.max()
            first_timestamp = result_df.index.min()
            
            print(f"Detecting Gold OHLCV data that was previously downloaded:")
            print(f" > latest timestamp available (UTC): {latest_timestamp}")
            print(f" > first timestamp available (UTC): {first_timestamp}")
            print("")
            
            # Get new data from the last available date
            ticker = yf.Ticker(symbol)
            new_data = ticker.history(start=latest_timestamp.strftime('%Y-%m-%d'), interval='1d')
            
            if not new_data.empty:
                # Keep only OHLCV columns and rename
                new_data = new_data[['Open', 'High', 'Low', 'Close', 'Volume']]
                new_data.columns = new_data.columns.str.lower()
                
                # Handle timezone for new data
                if new_data.index.tz is None:
                    new_data.index = new_data.index.tz_localize('UTC')
                
                # Combine existing and new data
                result_df = pd.concat([result_df, new_data])
                # Remove duplicates and sort
                result_df = result_df[~result_df.index.duplicated(keep='last')].sort_index()
        else:
            # Get all available data if no CSV exists
            ticker = yf.Ticker(symbol)
            result_df = ticker.history(start=start, interval='1d')
            
            if result_df.empty:
                print(f"No data retrieved for Gold")
                return None
                
            # Keep only OHLCV columns and rename
            result_df = result_df[['Open', 'High', 'Low', 'Close', 'Volume']]
            result_df.columns = result_df.columns.str.lower()
            
            # Handle timezone
            if result_df.index.tz is None:
                result_df.index = result_df.index.tz_localize('UTC')
        
        # Remove any NaN values
        result_df.dropna(inplace=True)
        
        # Save the data
        # Reset index to make Date a column, then save
        result_df.reset_index().rename(columns={'index': 'Date'}).to_csv(csv_path, index=False)
        result_df.to_json(f'Resources/{filename}_1D_yfinance.json', orient='records', date_format='iso')
        
        print(f"\nGold OHLCV data has been saved to disk and is available now =)")
        print("\nFirst few rows:")
        print(result_df.head())
        print("\nLast few rows:")
        print(result_df.tail())
        print("\nShape:", result_df.shape)
        
        return result_df
    
    except Exception as e:
        print(f"Error in download_and_merge_gold_OHLCV_yf: {str(e)}")
        return None

In [48]:
gold_data = download_and_merge_gold_OHLCV_yf(symbol='GC=F', start='2016-10-24')

Error in download_and_merge_gold_OHLCV_yf: 'Index' object has no attribute 'tz'


/var/folders/nb/f4s8fppj0j3gshk7xfn2cwf00000gn/T/ipykernel_54275/1039822623.py:14: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  result_df['Date'] = pd.to_datetime(result_df['Date'])


In [49]:
gold_data